In [1]:
using Pkg; Pkg.activate(".")

 Activating environment at `~/GitHub/problem-sets-1010/class-sets/09-09/Project.toml`


## Introduction to Programming in Julia

#### *11 September 2020*
#### *DATA 1010*

Today we will practice programming in Julia. If you are already familiar with Python, R, or MATLAB, you are likely to find Julia to be familiar. The primary distinction between Julia and those languages is that your code is compiled down to machine code behind the scenes, rather than being run in an interpreter. This has major implications for performance:

In [2]:
"""
Calculate the nth term Σₖ₌₁ⁿ(1/k) of the harmonic series
"""
function harmonic(n)
    s = 0.0
    for k in 1:n
        s += 1/k
    end
    s
end

harmonic

In [3]:
@time harmonic(10^8)

  0.095528 seconds


18.997896413852555

`@time` is an example of a **macro** in Julia. It allows us flexibility to do things with code other than just running it. You mostly won't need to think about the details of macros, but you will have occasion to use them.

Let's compare to Python. We can run Python in a Julia notebook using triple-quoted or single-quoted strings with `py` prefixed. 

In [4]:
using PyCall
py"""
def harmonic(n):
    s = 0.0
    for k in range(1,n+1):
        s += 1/k
    return s
"""
@time py"harmonic(10**8)"

  4.927401 seconds (3.34 k allocations: 172.670 KiB)


18.997896413852555

We can see the exact instructions being handed to the CPU for execution when we call a Julia function with `@code_native`. It isn't really readable, but the point is that there aren't many instructions. The amount of code involved in evaluating the analogous Python code is vastly larger.

In [5]:
@code_native harmonic(10^8)

	.section	__TEXT,__text,regular,pure_instructions
; ┌ @ In[2]:6 within `harmonic'
; │┌ @ range.jl:5 within `Colon'
; ││┌ @ range.jl:280 within `UnitRange'
; │││┌ @ range.jl:285 within `unitrange_last'
; ││││┌ @ operators.jl:350 within `>='
; │││││┌ @ int.jl:441 within `<='
	testq	%rdi, %rdi
; │└└└└└
	jle	L79
	movq	%rdi, %rax
	sarq	$63, %rax
	andnq	%rdi, %rax, %rax
; │ @ In[2]:7 within `harmonic'
	negq	%rax
	movl	$1, %ecx
	vxorpd	%xmm0, %xmm0, %xmm0
	movabsq	$5336010080, %rdx       ## imm = 0x13E0D0D60
	vmovsd	(%rdx), %xmm1           ## xmm1 = mem[0],zero
	nopl	(%rax,%rax)
; │┌ @ int.jl:92 within `/'
; ││┌ @ float.jl:277 within `float'
; │││┌ @ float.jl:262 within `AbstractFloat'
; ││││┌ @ float.jl:60 within `Float64'
L48:
	vcvtsi2sd	%rcx, %xmm3, %xmm2
; ││└└└
; ││ @ int.jl:92 within `/' @ float.jl:407
	vdivsd	%xmm2, %xmm1, %xmm2
; │└
; │┌ @ float.jl:401 within `+'
	vaddsd	%xmm2, %xmm0, %xmm0
; │└
; │┌ @ range.jl:624 within `iterate'
; ││┌ @ promotion.jl:398 within `=='
	leaq	(%rax,%rcx

The flip side of Julia's compilation is that a lot of work sometimes has to happen when packages are being loaded or when functions are being run for the first time. You should expect Python and R to be snappier when you're first loading up your session and you aren't doing anything heavy duty.

---

## Exercises

To make it easier to remember differences in syntax and common function names between Python and Julia, I recommend keeping a window open with the Julia-Python-R [cheatsheet](https://data1010.github.io/docs/cheatsheets/jpr-cheatsheet.pdf) (also linked from the course website). 

**Exercise 1**. Write a function with a single parameter `n` which outputs an `n × n` matrix like the one shown. (The cupcake ensemble was made for the famous mathematician and linear algebra educator [Gilbert Strang](http://www-math.mit.edu/~gs/)). 

![Tridiagonal matrix](cupcakes.png)

**Solution**.

---

**Exercise 2: Eigenvectors**. Consider a random symmetric $n \times n$ matrix $A$ defined by

```julia
    n = 10
    A = zeros(n,n)
    for i=1:n
        for j=1:i
            A[i,j] = rand() # returns a Unif([0,1])
            A[j,i] = A[i,j]
        end
    end
```

and define $\mathbf{v}_0 = [1,0,\ldots,0] \in \mathbb{R}^n$. For $k \geq 0$, define $\mathbf{v}_{k+1} = \frac{A\mathbf{v}_{k}}{|A\mathbf{v}_{k}|}$. Then as $k\to\infty$,  $\mathbf{v}_k$ converges to the eigenvector with the eigenvalue which is largest in absolute value.

Implement this algorithm in Julia and compare the eigenvector you find to the ones returned by Julia's `eigen` function. 

**Solution.**

---

### Pointwise operations (broadcasting)
In NumPy, operations are pointwise by default:

In [6]:
py"""
import numpy as np
A = np.array([[1,2],[3,4]])
"""
py"A*A"

2×2 Array{Int64,2}:
 1   4
 9  16

In Julia, operations correspond to their mathematical meaning by default, while pointwise operations are indicated with a dot:

In [7]:
A = [1 2
     3 4]
A .* A

2×2 Array{Int64,2}:
 1   4
 9  16

The same is true for function application: `f.([1,2,3])` is essentially equivalent to `[f(1),f(2),f(3)]`.

**Exercise 3**. Plot the function $f(x) = \sin^2(x) + 1$ over the interval $[0,2\pi]$ by applying $f$ to each element of the array `x`. Complete the lines below and uncomment them.

**Solution.**

In [8]:
using Plots
x = range(0, 2π, length=100)
#y = 
#plot(x,y, legend=false)

┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1278


0.0:0.06346651825433926:6.283185307179586

---

### Multiple dispatch

In math, we frequently re-use the same notation for different operations. For example, $2\cdot 3$ is real-number multiplication, while $\mathbf{v} \cdot \mathbf{w}$ probably refers to a vector dot product. 

In Julia, we can do the same thing:

In [9]:
mydot(a::Real, b::Real) = a * b
mydot(v::Vector, w::Vector) = sum([a*b for (a,b) in zip(v,w)])

mydot (generic function with 2 methods)

In [10]:
mydot(2,3)

6

In [11]:
mydot([1,2,3],[4,5,6])

32

This feature of Julia is called **multiple dispatch**. If you're looking for more video content for evening entertainment, there's an [excellent half-hour discussion](https://www.youtube.com/watch?v=kc9HwsxE1OY) of how multiple dispatch is key to Julia's success and how it compares to the design patterns would would use instead in Python.

You can define your own types in Julia (the type annotations are optional) like so: 

```julia
struct Line
    slope::Real
    intercept::Real
end

L = Line(1,2) # represents the line y = x + 2
L.slope # returns 1.0
L.intercept # returns 2.0
```

**Exercise 4**. Write a function `intersect` which returns the intersection point of two `Line` objects.

In [12]:
struct Line
    slope::Real
    intercept::Real
end

function intersection(L::Line,M::Line)
    # add code here
end

using Test
@test intersection(Line(1,1),Line(-1,2)) == (1,1)

intersection (generic function with 1 method)

---

### Plotting

**Exercise 5**. Approximately reproduce the figure below. 

Hint 1: check out the [Plots.jl cheatsheet](https://data1010.github.io/docs/cheatsheets/plotsjl-cheatsheet.pdf) to figure out what kind of visualization we're looking at.  
Hint 2: the data you want to pass are x (a vector of x values), `y` (a vector of y values) and `z` (a *two*-dimensional array). To make a 2D array, use the two-dimensional array comprehension; e.g. `[f(x,y) for x in 1:10, y in 1:10]` returns a 100-entry matrix whose $(x,y)$ entry is $f(x,y)$.

![](heatmap.svg)

---

**Exercise 6** (challenge problem). A *fixed point* of a function $f$ is a domain element $x$ satisfying $f(x) = x$. Banach's **fixed point theorem** says that if $f$ satisfies the property that for some $K < 1$, we have $|f(x) - f(y)| \leq K |x-y|$ for all $x$ and $y$ in the domain of $f$, then $f$ has a unique fixed point. Furthermore, this fixed point is the limit of the sequence $x_0$, $f(x_0)$, $f(f(x_0))$, $f(f(f(x_0)))\ldots$. 

Devise a scheme for using the fixed point theorem to solve the matrix equation $A\mathbf{x} = \mathbf{b}$ for $\mathbf{x}$, where $A$ is a matrix, subject to constraints that you may stipulate, and $\mathbf{b}$ is a vector.